# VAR Model

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR

In [ ]:
with open('../sql-scripts/var.txt', 'r') as sql_file:
  sql_script = sql_file.read()

conn = sqlite3.connect('orlando.db')
var_df = pd.read_sql_query(sql_script, conn)
conn.close()

In [ ]:
# Drop rows with missing values since VAR requires complete data
var_df = var_df.dropna()

# Select relevant columns for the VAR model
columns_for_var = ['zillow', 'income', 'unemployment', 'distance', 'duration']
var_df = var_df[columns_for_var]

# Fit the VAR model
model = VAR(var_df)
results = model.fit()

# Print the summary of the model
print(results.summary())

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 21, Apr, 2025
Time:                     17:05:16
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                    33.3023
Nobs:                     357.000    HQIC:                   33.1060
Log likelihood:          -8389.09    FPE:                2.09639e+14
AIC:                      32.9764    Det(Omega_mle):     1.92877e+14
--------------------------------------------------------------------
Results for equation zillow
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const               68389.506807     26467.390840            2.584           0.010
L1.zillow               0.882260         0.038577           22.870           0.000
L1.income              -1.479204         0.350244  

c:\Users\brad\miniconda3\envs\ds2025-group3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)


# Prophet Model

In [14]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import cmdstanpy
cmdstanpy.install_cmdstan(compiler=True)

CmdStan install directory: C:\Users\brad\.cmdstan
Installing RTools40
C++ toolchain 'RTools' version: 4.0
Install directory: C:\Users\brad\.cmdstan
Download successful, file: RTools40.exe
Installing the C++ toolchain: RTools40
RTools40.exe /SP- /SILENT /SUPPRESSMSGBOXES /CURRENTUSER LANG="English" /DIR="Users\brad\.cmdstan\RTools40" /NOICONS /NORESTART
Installed RTools40
pacman -Sy mingw-w64-x86_64-make --noconfirm


17:50:31 - cmdstanpy - INFO - Add C++ toolchain to $PATH: C:\Users\brad\.cmdstan\RTools40


Installed mingw32-make.exe
Installing CmdStan version: 2.36.0
Download successful, file: C:\Users\brad\AppData\Local\Temp\tmp1c583rma
Extracting distribution
Unpacked download as cmdstan-2.36.0
Building version cmdstan-2.36.0, may take several minutes, depending on your system.
Installed cmdstan-2.36.0
Test model compilation


True

In [ ]:
with open('../sql-scripts/prophet.txt', 'r') as sql_file:
  sql_script = sql_file.read()

conn = sqlite3.connect('orlando.db')
prophet_df = pd.read_sql_query(sql_script, conn)
conn.close()

In [16]:
# Ensure your date column is in datetime format and rename columns for Prophet
prophet_df["ds"] = pd.to_datetime(prophet_df["period"])
prophet_df = prophet_df.rename(columns={'value': 'y'})

# Define a cutoff date for training/test split
cutoff_date = "2023-01-01"

# Container to hold results for each neighborhood
results = []

# Loop over each neighborhood
for neighborhood, group_df in prophet_df.groupby("neighborhood"):
  # Sort by date
  group_df = group_df.sort_values("ds")

  # Split the data: training before cutoff, test starting from cutoff
  train_df = group_df[group_df["ds"] < cutoff_date].copy()
  test_df  = group_df[group_df["ds"] >= cutoff_date].copy()

  # Skip neighborhoods with insufficient training or testing data
  if train_df["y"].notna().sum() < 2 or test_df["y"].notna().sum() < 1:
    print(f"Skipping {neighborhood} due to insufficient data.")
    continue

  # Fit Prophet on the training data
  m = Prophet()
  m.fit(train_df)

  # Forecast 10 years into the future
  future_10yr = m.make_future_dataframe(periods=120, freq="M")
  forecast_10yr = m.predict(future_10yr)

  # Collect results
  results.append({
    "neighborhood": neighborhood,
    "forecast_10yr": forecast_10yr
  })

# Example: Access forecast for a specific neighborhood
for result in results:
  if result["neighborhood"] == "Windhover":
    print(result["forecast_10yr"].head())

18:00:07 - cmdstanpy - INFO - Chain [1] start processing
18:00:08 - cmdstanpy - INFO - Chain [1] done processing
18:00:08 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:08 - cmdstanpy - INFO - Chain [1] start processing
18:00:08 - cmdstanpy - INFO - Chain [1] done processing
18:00:08 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=10790 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\v7_bwt5a.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\buifexen.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_model1lx12yw2\prophet_model-20250421180008.csv method=optimize algorithm=newton iter=10000' failed: 

In [17]:
# Define an evaluation function for the test set forecast
def evaluate_forecast(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # expressed as percentage
    return {"MAE": mae, "RMSE": rmse, "MAPE": mape}

# Ensure your date column is in datetime format and rename columns for Prophet
prophet_df["ds"] = pd.to_datetime(prophet_df["period"])
prophet_df = prophet_df.rename(columns={'value': 'y'})

# Define a cutoff date for training/test split (adjust as needed)
cutoff_date = "2023-01-01"

# Container to hold results for each neighborhood
results = []

# Loop over each neighborhood (using "neighborhood")
for neighborhood, group_df in prophet_df.groupby("neighborhood"):
    # Sort by date (the "ds" column)
    group_df = group_df.sort_values("ds")

    # Split the data: training before cutoff, test starting from cutoff
    train_df = group_df[group_df["ds"] < cutoff_date].copy()
    test_df  = group_df[group_df["ds"] >= cutoff_date].copy()

    # Skip neighborhoods with insufficient training or testing data
    if train_df["y"].notna().sum() < 2 or test_df["y"].notna().sum() < 1:
        print(f"Skipping {neighborhood} due to insufficient data: "
              f"train count = {train_df['y'].notna().sum()}, "
              f"test count = {test_df['y'].notna().sum()}")
        continue

    # Fit Prophet on the training data
    m = Prophet()
    try:
        m.fit(train_df)
    except Exception as e:
        print(f"Error fitting Prophet for {neighborhood}: {e}")
        continue

    # --- Evaluate on Test Data ---
    # Create a future dataframe covering the test period (same number of periods as in test)
    future_test = m.make_future_dataframe(periods=len(test_df), freq="M")
    forecast_test = m.predict(future_test)

    # Select only the forecasted values in the test period (using the cutoff date)
    forecast_test = forecast_test[forecast_test["ds"] >= cutoff_date]
    y_true = test_df["y"].values
    y_pred = forecast_test["yhat"].values
    metrics_dict = evaluate_forecast(y_true, y_pred)

    # --- Forecast 10 Years into the Future ---
    future_10yr = m.make_future_dataframe(periods=120, freq="M")
    forecast_10yr = m.predict(future_10yr)

    # --- Forecast 15 Years into the Future ---

    future_15yr = m.make_future_dataframe(periods=180, freq="M")
    forecast_15yr = m.predict(future_15yr)

    # Collect results for the current region
    results.append({
        "neighborhood": neighborhood,
        "metrics": metrics_dict,
        "forecast_test": forecast_test,
        "forecast_10yr": forecast_10yr,     # Full forecast including history plus 10 years
        "forecast_15yr": forecast_15yr      # Full forecast including history plus 15 years
    })

# Convert the results list to a DataFrame of metrics (for summary, if desired)
metrics_df = pd.DataFrame([{"neighborhood": r["neighborhood"], **r["metrics"]} for r in results])
print("Evaluation Metrics:")
print(metrics_df.sort_values("MAE").head(10))

18:00:20 - cmdstanpy - INFO - Chain [1] start processing
18:00:20 - cmdstanpy - INFO - Chain [1] done processing
18:00:20 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:20 - cmdstanpy - INFO - Chain [1] start processing
18:00:20 - cmdstanpy - INFO - Chain [1] done processing
18:00:20 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
18:00:20 - cmdstanpy - INFO - Chain [1] start processing
18:00:20 - cmdstanpy - INFO - Chain [1] done processing
18:00:20 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:20 - cmdstanpy - INFO - Chain [1] start processing
18:00:20 - cmdstanpy - INFO - Chain [1] done processing


Error fitting Prophet for Airport North: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=87975 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\6fgfjzqa.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\59jot07f.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_model97rec8bl\prophet_model-20250421180020.csv method=optimize algorithm=newton iter=10000' failed: 


18:00:20 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
18:00:20 - cmdstanpy - INFO - Chain [1] start processing
18:00:20 - cmdstanpy - INFO - Chain [1] done processing
18:00:20 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:20 - cmdstanpy - INFO - Chain [1] start processing
18:00:20 - cmdstanpy - INFO - Chain [1] done processing
18:00:20 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


Error fitting Prophet for Alafaya: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=18363 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\9lz5m8gg.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\lidvosjr.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_modelt47dwj6y\prophet_model-20250421180020.csv method=optimize algorithm=newton iter=10000' failed: 
Error fitting Prophet for Audubon Park: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=67032 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\9wai7j64.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\tcru_atb.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_modely7rqvq2d\prophet_model-20250421180020.csv method=optimize algorithm=newton iter=10000' failed: 


18:00:21 - cmdstanpy - INFO - Chain [1] start processing
18:00:21 - cmdstanpy - INFO - Chain [1] done processing
18:00:21 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:21 - cmdstanpy - INFO - Chain [1] start processing
18:00:21 - cmdstanpy - INFO - Chain [1] done processing
18:00:21 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
18:00:21 - cmdstanpy - INFO - Chain [1] start processing
18:00:21 - cmdstanpy - INFO - Chain [1] done processing
18:00:21 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:21 - cmdstanpy - INFO - Chain [1] start processing
18:00:21 - cmdstanpy - INFO - Chain [1] done processing
18:00:21 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


...

18:00:35 - cmdstanpy - INFO - Chain [1] done processing
18:00:35 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
18:00:36 - cmdstanpy - INFO - Chain [1] start processing
18:00:36 - cmdstanpy - INFO - Chain [1] done processing
18:00:36 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:36 - cmdstanpy - INFO - Chain [1] start processing
18:00:36 - cmdstanpy - INFO - Chain [1] done processing
18:00:36 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


Error fitting Prophet for South Eola: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=53620 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\xl2dg1r7.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\a0oix7_t.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_modelu70ujo0o\prophet_model-20250421180035.csv method=optimize algorithm=newton iter=10000' failed: 
Error fitting Prophet for South Orange: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=60103 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\v90yfv5l.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\9e0mzls_.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_modelc8bkbruy\prophet_model-20250421180036.csv method=optimize algorithm=newton iter=10000' failed: 


18:00:36 - cmdstanpy - INFO - Chain [1] start processing
18:00:36 - cmdstanpy - INFO - Chain [1] done processing
18:00:36 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:36 - cmdstanpy - INFO - Chain [1] start processing
18:00:36 - cmdstanpy - INFO - Chain [1] done processing
18:00:36 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
18:00:36 - cmdstanpy - INFO - Chain [1] start processing
18:00:36 - cmdstanpy - INFO - Chain [1] done processing
18:00:36 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
18:00:36 - cmdstanpy - INFO - Chain [1] start processing
18:00:36 - cmdstanpy - INFO - Chain [1] done processing
18:00:36 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


Error fitting Prophet for South Semoran: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=38239 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\i9k3i4oo.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\1m69veou.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_modelanlyymin\prophet_model-20250421180036.csv method=optimize algorithm=newton iter=10000' failed: 
Error fitting Prophet for Southchase: Error during optimization! Command 'C:\Users\brad\miniconda3\envs\ds2025-group3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=27384 data file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\cf6ghhs7.json init=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\n_qh1jm0.json output file=C:\Users\brad\AppData\Local\Temp\tmp33eg6jy6\prophet_modelsq5yc4t8\prophet_model-20250421180036.csv method=optimize algorithm=newton iter=10000' failed: 


KeyboardInterrupt: 